In [2]:
from __future__ import absolute_import, division, print_function
import uproot
import numpy as np
import matplotlib.pyplot as plt

# energyflow imports
import energyflow as ef
from energyflow.archs import *
from energyflow.utils import data_split, remap_pids, to_categorical

from sklearn.metrics import roc_auc_score, roc_curve

In [10]:
h2gg_dir = '/home/sambt/pythia-samples/optimal-classifiers/H2gg-ee/'
h2qq_dir = '/home/sambt/pythia-samples/optimal-classifiers/H2qqbar-ee/'
kern = -1
CFs = [0.0001,1/3,2/3,1,4/3,5/3,2,7/3,8/3,3]
CA = 3.0

In [11]:
for CF in CFs:
    fname_gg = 'kernel{0}/H2gg-ee-100k_CF_{1:.1f}_CA_{2:.1f}/total.root'.format(kern,CF,CA)
    fname_qq = 'kernel{0}/H2qqbar-ee-100k_CF_{1:.1f}_CA_{2:.1f}/total.root'.format(kern,CF,CA)
    
    f_gg = uproot.open(h2gg_dir+fname_gg)['EventTree']
    f_qq = uproot.open(h2qq_dir+fname_qq)['EventTree']